# Testing GraphQL Handler

This notebook is for testing the GraphQL handler functionality with LiveHeats API.

In [ ]:
import os
import json
import asyncio
import logging
from pathlib import Path
from typing import Dict, Any
from dotenv import load_dotenv

# Configure logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

# Add the ingestion package to the path
import sys
sys.path.append('../src')

from ingestion.handlers.graphql.graphql_handler import GraphQLHandler

## Load Environment Variables

First, let's load the environment variables from .env.liveheats

In [ ]:
# Load environment variables
env_path = Path(__file__).parents[2] / ".platform_config" / "dev_platform" / ".env.liveheats"
load_dotenv(env_path)

# Print current scope
print(f"Current scope from env: {os.environ.get('SCOPE', 'Not set')}")

## Set Up Test Configuration

Now let's set up our test configuration that matches the format in config.yaml

In [ ]:
task_config = {
    "query": """
        query OrganisationAthletes($id: ID!, $page: Int!, $per: Int!) {
            organisationAthletes(id: $id, page: $page, per: $per) {
                athletes {
                    id
                    name
                    dob
                    nationality
                    properties
                    memberships {
                        id
                    }
                }
                totalCount
            }
        }
    """,
    "variables": {
        "id": "1",  # This should be a valid organization ID
        "page": 1,
        "per": 100
    },
    "sink": {
        "type": "local",  # Use local file system instead of S3 for testing
        "key_prefix": "lh_organisations"
    }
}

# Set up mock secrets
required_secrets = [
    "CLIENT_ID",
    "CLIENT_SECRET",
    "GRAPHQL_URL",
    "AUTH_URL",
    "SCOPE"
]

# Verify all required secrets are in environment
missing = [s for s in required_secrets if s not in os.environ]
if missing:
    raise ValueError(f"Missing required environment variables: {missing}")

mock_secrets = {
    "CLIENT_ID": os.environ["CLIENT_ID"],
    "CLIENT_SECRET": os.environ["CLIENT_SECRET"],
    "GRAPHQL_URL": os.environ["GRAPHQL_URL"],
    "AUTH_URL": os.environ["AUTH_URL"],
    "SCOPE": os.environ["SCOPE"]
}

# Print the secrets (hiding sensitive values)
safe_secrets = mock_secrets.copy()
safe_secrets["CLIENT_ID"] = safe_secrets["CLIENT_ID"][:10] + "..."
safe_secrets["CLIENT_SECRET"] = safe_secrets["CLIENT_SECRET"][:10] + "..."
print("Mock secrets:")
print(json.dumps(safe_secrets, indent=2))

## Initialize Handler and Test Authentication

Let's first test just the authentication part

In [ ]:
# Set up temporary data directory
data_dir = Path("/tmp/graphql_test")
data_dir.mkdir(parents=True, exist_ok=True)
os.environ["DATA_DIR"] = str(data_dir)

# Initialize handler
handler = GraphQLHandler(task_config)

# Test authentication
async def test_auth():
    try:
        token = await handler._get_auth_token(mock_secrets)
        print("Successfully got auth token")
        print(f"Token starts with: {token[:20]}...")
    except Exception as e:
        print(f"Auth failed: {str(e)}")

await test_auth()

## Test Full Query Execution

Now let's test the full query execution

In [ ]:
# Test full query execution
async def test_query():
    try:
        result = await handler.execute_query(mock_secrets)
        print("Query succeeded!")
        print("Result:")
        print(json.dumps(result, indent=2))
    except Exception as e:
        print(f"Query failed: {str(e)}")

await test_query()

## Test Full Data Processing

Finally, let's test the full data processing pipeline

In [ ]:
# Test full data processing
async def test_process_data():
    try:
        await handler.process_data()
        print("Data processing succeeded!")
        
        # Check output file
        output_files = list(data_dir.glob("**/*.json"))
        print(f"\nOutput files:")
        for f in output_files:
            print(f"- {f}")
            
            # Print first few lines of the file
            with open(f) as fp:
                data = json.load(fp)
                print("\nFirst few lines of data:")
                print(json.dumps(data, indent=2)[:500] + "...")
    except Exception as e:
        print(f"Data processing failed: {str(e)}")

await test_process_data()